## Dependencies

In [ ]:
# Dependencies
import pandas as pd
import numpy as np
from datetime import date
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error, r2_score
import xgboost as xgb
import matplotlib.pyplot as plt
import json

In [ ]:
# Path to the Excel file
excel_file = f'train_data.xlsx'

# List of Retailers
sheet_names = ['Walmart', 'LCL', "Amazon"]  

# Initialize an empty DataFrame for merging
merged_df = pd.DataFrame()

# Read each sheet into a DataFrame and merge them
for retailer in sheet_names:
    df = pd.read_excel(excel_file, sheet_name=retailer, index_col='Date')
    df.index = pd.to_datetime(df.index)
    df.columns = [f'{retailer} | {col}' for col in df.columns]
    # Merge with the accumulated DataFrame
    if merged_df.empty:
        merged_df = df
    else:
        merged_df = pd.merge(merged_df, df, left_index=True, right_index=True, how='left')

df = merged_df.copy()

#set up the target var
sales = [sku for sku in df.columns if 'Pricing' not in sku]
price_features = [sku for sku in df.columns if 'Pricing' in sku]
temporal_features = [
    'quarter',
    'month',
    'year',
    'weekofyear',
    'weekofmonth'
]

# Spend
spend_df = pd.read_excel(excel_file, sheet_name="Spend", index_col='Date')
spend_df.index = pd.to_datetime(spend_df.index)
spend_list = spend_df.columns

# Add media features to the dataframe
media_df = pd.read_excel(excel_file, sheet_name='Impressions', index_col='Date')
media_df.index = pd.to_datetime(df.index)
### filter the media thats less than 10k spend
droplist=[]
for media in media_df.columns:
    media_spend = media.replace("Impressions", "Spend")
    if media_spend in spend_list and spend_df[media_spend].sum()<8000:
        droplist.append(media)

droplist.extend(['PR | LIV - PR - Mailer - Impressions', 
                 'PR | LIV - PR - Activation Content - Impressions', 
                 'PR | LIV - PR - Event Brand Posts - Impressions',
                 'PR | LIV - PR - Merch Drop Coverage - Impressions'])

media_df.drop(columns=droplist, inplace=True)
######
cat_features_withoutlag = media_df.columns
df = pd.merge(df, media_df, left_index=True, right_index=True, how='left')

cat_features_withoutlag

Index(['Social | LiquidIV | Vitamins, Minerals & Supplements | Meta - Impressions',
       'Social | LiquidIV | Vitamins, Minerals & Supplements | Pinterest - Impressions',
       'Social | LiquidIV | Vitamins, Minerals & Supplements | TikTok - Impressions',
       'Programmatic | LiquidIV | Vitamins & Supplements | DV360 | Video - Impressions',
       'Programmatic | LiquidIV | Vitamins & Supplements | TTD | Display - Impressions',
       'Programmatic | LiquidIV | Vitamins & Supplements | TTD | Video - Impressions',
       'Google Ads | LiquidIV | Vitamins and Supplements - Impressions',
       'DirectIO | LiquidIV | Vitamins | Samsung Ads - Impressions',
       'Sample | LIV - Event - Sampling',
       'PI | LIV - Paid Influencer - Boosted impressions',
       'PR | LIV - PR - Media Coverage - Total Impressions',
       'PR | LIV - PR - Paid Influencer - Organic - Impressions',
       'AMZ | Liquid IV_Hydration Packets_Brand - Impressions',
       'AMZ | Liquid IV_Hydration Packets_

In [ ]:
# mylist = []
# for media in media_df.columns:
#     media_spend = media.replace("Impressions", "Spend")
#     if media_spend in spend_list:
#         mylist.append(int(spend_df[media_spend].sum()))
#     else:
#         mylist.append(np.nan)
# new_df = pd.DataFrame()
# new_df["Media"] = media_df.columns
# new_df["Total Spend"] = mylist
# new_df.to_excel("temp.xlsx", index=False)
# new_df

In [ ]:
len(droplist)/(len(droplist)+len(media_df.columns))

0.5384615384615384

In [ ]:
price_features

['Walmart | Liquid IV Lemon Lime 12p 10ct - Pricing',
 'Walmart | Liquid IV Strawberry 12p 10ct - Pricing',
 'Walmart | Liquid IV Passion Fruit 12p 6ct - Pricing',
 'LCL | Liquid IV Lemon Lime 12p 6ct - Pricing',
 'LCL | Liquid IV Strawberry 8p 16ct - Pricing',
 'LCL | Liquid IV Passion Fruit 12p 6ct - Pricing',
 'Amazon | LiquidIV - Pricing']

### hyperparameters

In [ ]:
# Access the environment variable, enter the category here
CATEGORY = 'LiquidIV'
portion = 0.8
params = {
            "base_score": 0.5,
            "booster": "gbtree",
            "n_estimators": 3000,
            "early_stopping_rounds": 150,
            "objective": "reg:absoluteerror",
            "max_depth": 5,
            "learning_rate": 0.01,
            "colsample_bytree": 0.9,
            "reg_alpha": 0.1,
            "reg_lambda": 10
        }
cat_list = cat_features_withoutlag
print(CATEGORY)

LiquidIV


In [ ]:
df

,Walmart | Liquid IV Lemon Lime 12p 10ct,Walmart | Liquid IV Lemon Lime 12p 10ct - Pricing,Walmart | Liquid IV Strawberry 12p 10ct,Walmart | Liquid IV Strawberry 12p 10ct - Pricing,Walmart | Liquid IV Passion Fruit 12p 6ct,Walmart | Liquid IV Passion Fruit 12p 6ct - Pricing,LCL | Liquid IV Lemon Lime 12p 6ct,LCL | Liquid IV Lemon Lime 12p 6ct - Pricing,LCL | Liquid IV Strawberry 8p 16ct,LCL | Liquid IV Strawberry 8p 16ct - Pricing,...,Sample | LIV - Event - Sampling,PI | LIV - Paid Influencer - Boosted impressions,PR | LIV - PR - Media Coverage - Total Impressions,PR | LIV - PR - Paid Influencer - Organic - Impressions,AMZ | Liquid IV_Hydration Packets_Brand - Impressions,AMZ | Liquid IV_Hydration Packets_Product - Impressions,AMZ | Liquid IV_Hydration Packets_Defense - Impressions,AMZ | Liquid IV_Hydration Packets_StoreKW_SB - Impressions,AMZ | Liquid IV_Hydration Packets_StorePT_SB - Impressions,AMZ | SP_Unbranded_Research_Phrase_French_Liquid IV - Impressions
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2023-01-15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2023-01-22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2023-01-29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
2023-02-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-09-01,12710.91,19.959193,10001.05,19.904433,3548.38,13.970000,3870.23,14.157662,3614.79,15.279022,...,6510.0,0,0,0,150,0,0,0,0,33
2024-09-08,12054.02,19.934452,8996.99,19.955279,3041.74,13.899565,3840.44,14.882922,1539.00,34.699667,...,0.0,0,0,0,55,0,0,0,0,14
2024-09-15,11643.64,19.944768,10966.79,19.940736,3450.59,13.970000,3751.29,14.915828,1625.53,33.879167,...,0.0,0,0,0,0,0,0,0,0,1813


## Forecaster

#### Load the Data

#### Add temporal features and lagged features

In [ ]:
# Extract temporal features
df['quarter'] = df.index.quarter
df['month'] = df.index.month
df['year'] = df.index.year
df['weekofyear'] = df.index.isocalendar().week
df['weekofmonth'] = df.index.to_series().apply(lambda x: (x.day - 1) // 7 + 1)
#create lag features
lag = 2
cat_features_withlag = []
for f in cat_features_withoutlag:
    cat_features_withlag.append(f)
    for l in range(1, lag+1):
        df[f'{f}_lag_{l}'] = df[f].shift(l)
        cat_features_withlag.append(f'{f}_lag_{l}')

In [ ]:
# Dimension reduction using the selected media features - "cat_features_withlag"
df = df[temporal_features+sales+price_features+cat_features_withlag]
df.head(3)

,quarter,month,year,weekofyear,weekofmonth,Walmart | Liquid IV Lemon Lime 12p 10ct,Walmart | Liquid IV Strawberry 12p 10ct,Walmart | Liquid IV Passion Fruit 12p 6ct,LCL | Liquid IV Lemon Lime 12p 6ct,LCL | Liquid IV Strawberry 8p 16ct,...,AMZ | Liquid IV_Hydration Packets_Defense - Impressions_lag_2,AMZ | Liquid IV_Hydration Packets_StoreKW_SB - Impressions,AMZ | Liquid IV_Hydration Packets_StoreKW_SB - Impressions_lag_1,AMZ | Liquid IV_Hydration Packets_StoreKW_SB - Impressions_lag_2,AMZ | Liquid IV_Hydration Packets_StorePT_SB - Impressions,AMZ | Liquid IV_Hydration Packets_StorePT_SB - Impressions_lag_1,AMZ | Liquid IV_Hydration Packets_StorePT_SB - Impressions_lag_2,AMZ | SP_Unbranded_Research_Phrase_French_Liquid IV - Impressions,AMZ | SP_Unbranded_Research_Phrase_French_Liquid IV - Impressions_lag_1,AMZ | SP_Unbranded_Research_Phrase_French_Liquid IV - Impressions_lag_2
Date,,,,,,,,,,,,,,,,,,,,,
2023-01-08,1,1,2023,1,2,NaN,NaN,NaN,NaN,NaN,...,NaN,0,NaN,NaN,0,NaN,NaN,0,NaN,NaN
2023-01-15,1,1,2023,2,3,NaN,NaN,NaN,NaN,NaN,...,NaN,0,0.0,NaN,0,0.0,NaN,0,0.0,NaN
2023-01-22,1,1,2023,3,4,NaN,NaN,NaN,NaN,NaN,...,0.0,0,0.0,0.0,0,0.0,0.0,0,0.0,0.0


In [ ]:
# the total volumn of sales for the category you selected
df[sales].sum().sum()

2824744.1499999966

## train

In [ ]:
# List to store individual models
models = []
results = []

# Training loop for each output
for i in range(len(sales)):
    # train test split
    FEATURES = temporal_features + [price_features[i]] + cat_features_withlag
    TARGET = sales[i]
    df_copy = df.dropna(subset=[TARGET]+FEATURES) #subset=[TARGET, price_features[i]]
    print(df.shape[0] - df_copy.shape[0]) #how many data rows are dropped
   
    # Drop rows with NaN values to get the valid sales data
    valid_sales_df = df[TARGET].dropna()
    split_index = int(len(valid_sales_df) * portion)# Determine the split index based on 75% for training and 25% for testing
    split_date = valid_sales_df.index[split_index]

    train = df_copy.loc[df_copy.index < split_date]
    test = df_copy.loc[df_copy.index >= split_date]

    X_train = train[FEATURES]
    y_train = train[TARGET]
    X_test = test[FEATURES]
    y_test = test[TARGET]
    
    monotonic_constraints = tuple([1 if feature in cat_features_withlag else 0 for feature in FEATURES])

    reg = xgb.XGBRegressor(**params,
                       monotone_constraints=monotonic_constraints
                       )
    
    reg.fit(
        X_train, y_train,
        eval_set=[(X_test, y_test)],
        verbose=0
    )
    
    # Save the model to local
    # reg.save_model(f'../DoveModel/{CATEGORY}/model_{i+1}.json')

    models.append([reg, FEATURES])

    # forecasting
    def mape(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    
    y_pred = reg.predict(X_test)
    y_true = y_test
    results.append(mape(y_true, y_pred))

61
61
70
77
78
77
29


In [ ]:
# Predict on test set
print(np.mean(results))
print(len(results))

14.246372904413947
7


## Inference

### Yearly ROI

In [ ]:
# yearly_df = pd.DataFrame(columns=["Year", CATEGORY, "Revenue", "Spend"])
# for year, group in df.groupby(pd.Grouper(freq="YE")):
#     year= year.year
#     TIME_START = group.index.min()
#     TIME_END = group.index.max()
    
#     # merge spending data to the df
#     merged_df = df.copy()
#     min_df = merged_df.min()
#     merged_df = merged_df.loc[TIME_START:TIME_END+pd.DateOffset(14)] # select defined time range

#     for media in cat_features_withoutlag:
#         X_ori = merged_df.copy()
#         X_zero = merged_df.copy()
#         # #get min
#         minimum_value = 0 #min_df[media]
#         # print(minimum_value)
#         X_zero.loc[TIME_START:TIME_END, f"{media}"] = minimum_value
#         X_zero.loc[TIME_START+pd.DateOffset(7):TIME_END+pd.DateOffset(7), f"{media}_lag_1"] = minimum_value
#         X_zero.loc[TIME_START+pd.DateOffset(14):TIME_END+pd.DateOffset(14), f"{media}_lag_2"] = minimum_value
#         revenue_list = []
#         for i, model in enumerate(models):
#             model, features = model
#             X_ori_copy = X_ori.dropna(subset=features)
#             X_zero_copy = X_zero.dropna(subset=features)
            
#             original_pred = model.predict(X_ori_copy[features])
#             zero_pred = model.predict(X_zero_copy[features])
#             revenue_list.append(sum(np.maximum(original_pred-zero_pred, 0))) # return on sku level
#         revenue = sum(revenue_list)
#         spend_col = media.replace("Impressions", "Spend")
#         if spend_col not in spend_list:
#             yearly_df.loc[len(yearly_df)] = [year, media, revenue, np.nan]
#         else:
#             media_spend = spend_df.loc[TIME_START:TIME_END, spend_col].sum()
#             yearly_df.loc[len(yearly_df)] = [year, media, revenue, media_spend]

# # Add a new column called ROI
# yearly_df['ROI'] = yearly_df.apply(lambda row: row['Revenue'] / row['Spend'] if row['Spend'] not in [None, 0] else np.nan, axis=1)
# yearly_df = yearly_df.sort_values(by='Spend', ascending=False)

# # Insert the new columns and drop the old one
# split_cols = yearly_df[CATEGORY].str.split(r' \| ', n=1, expand=True)
# yearly_df.insert(1, 'Category', split_cols[0])
# yearly_df[CATEGORY] = split_cols[1]
# yearly_df

In [ ]:
# yearly_df.to_excel("liquidIV_xgb_output.xlsx", index=False)

### Total ROI

In [ ]:
yearly_df = pd.DataFrame(columns=[CATEGORY, "Revenue", "Spend"])
TIME_START = pd.to_datetime("2023-01-01")
TIME_END = pd.to_datetime("2025-01-01")
    
# merge spending data to the df
merged_df = df.copy()
min_df = merged_df.min()
merged_df = merged_df.loc[TIME_START:TIME_END+pd.DateOffset(14)] # select defined time range

for media in cat_features_withoutlag:
    X_ori = merged_df.copy()
    X_zero = merged_df.copy()
    # #get min
    minimum_value = 0 #min_df[media]
    # print(minimum_value)
    X_zero.loc[TIME_START:TIME_END, f"{media}"] = minimum_value
    X_zero.loc[TIME_START+pd.DateOffset(7):TIME_END+pd.DateOffset(7), f"{media}_lag_1"] = minimum_value
    X_zero.loc[TIME_START+pd.DateOffset(14):TIME_END+pd.DateOffset(14), f"{media}_lag_2"] = minimum_value
    revenue_list = []
    for i, model in enumerate(models):
        model, features = model
        X_ori_copy = X_ori.dropna(subset=features)
        X_zero_copy = X_zero.dropna(subset=features)
        
        original_pred = model.predict(X_ori_copy[features])
        zero_pred = model.predict(X_zero_copy[features])
        revenue_list.append(sum(np.maximum(original_pred-zero_pred, 0))) # return on sku level
    revenue = sum(revenue_list)
    spend_col = media.replace("Impressions", "Spend")
    if spend_col not in spend_list:
        yearly_df.loc[len(yearly_df)] = [media, revenue, np.nan]
    else:
        media_spend = spend_df.loc[TIME_START:TIME_END, spend_col].sum()
        yearly_df.loc[len(yearly_df)] = [media, revenue, media_spend]

# Add a new column called ROI
yearly_df['ROI'] = yearly_df.apply(lambda row: row['Revenue'] / row['Spend'] if row['Spend'] not in [None, 0] else np.nan, axis=1)
yearly_df = yearly_df.sort_values(by='Spend', ascending=False)

# Insert the new columns and drop the old one
split_cols = yearly_df[CATEGORY].str.split(r' \| ', n=1, expand=True)
yearly_df.insert(0, 'Category', split_cols[0])
yearly_df[CATEGORY] = split_cols[1]
yearly_df

,Category,LiquidIV,Revenue,Spend,ROI
0,Social,"LiquidIV | Vitamins, Minerals & Supplements | ...",33604.679321,820074.810000,0.040978
2,Social,"LiquidIV | Vitamins, Minerals & Supplements | ...",2958.073730,351123.250000,0.008425
6,Google Ads,LiquidIV | Vitamins and Supplements - Impressions,115498.620056,136440.800000,0.846511
5,Programmatic,LiquidIV | Vitamins & Supplements | TTD | Vide...,0.000000,129235.813581,0.000000
1,Social,"LiquidIV | Vitamins, Minerals & Supplements | ...",793.058777,109341.493210,0.007253
4,Programmatic,LiquidIV | Vitamins & Supplements | TTD | Disp...,14.732544,108843.468111,0.000135
3,Programmatic,LiquidIV | Vitamins & Supplements | DV360 | Vi...,1357.565552,92131.240387,0.014735
7,DirectIO,LiquidIV | Vitamins | Samsung Ads - Impressions,242.131775,78272.365000,0.003093
14,AMZ,Liquid IV_Hydration Packets_Defense - Impressions,23672.873451,31514.360000,0.751177
12,AMZ,Liquid IV_Hydration Packets_Brand - Impressions,187209.205811,13409.420000,13.961022


### ROI By Cat

In [ ]:
# Initialize an empty DataFrame
group_df = pd.DataFrame(columns=["Category", "Revenue", "Spend", "ROI"])

# Group by category and sum revenue and spend
for category, category_df in yearly_df.groupby('Category'):
    if category == "PR":
        group_df = pd.concat([group_df, category_df[["Category", "Revenue", "Spend"]]], ignore_index=True)
    else:
        new_df = category_df[["Category", "Revenue", "Spend"]].sum()
        new_df["Category"] = category
        new_df["ROI"] = new_df['Revenue'] / new_df['Spend'] if new_df['Spend'] != 0 else np.nan
        group_df = pd.concat([group_df, pd.DataFrame([new_df])], ignore_index=True)

group_df

C:\Users\Eric.Yu\AppData\Local\Temp\ipykernel_11412\1322984423.py:12: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  group_df = pd.concat([group_df, pd.DataFrame([new_df])], ignore_index=True)


,Category,Revenue,Spend,ROI
0,AMZ,1.016043e+06,8.809433e+04,11.533583
1,DirectIO,2.421318e+02,7.827236e+04,0.003093
2,Google Ads,1.154986e+05,1.364408e+05,0.846511
3,PI,8.297510e+02,0.000000e+00,NaN
4,PR,6.333203e+01,NaN,NaN
5,PR,6.128487e+04,NaN,NaN
6,Programmatic,1.372298e+03,3.302105e+05,0.004156
7,Sample,2.508338e+04,0.000000e+00,NaN
8,Social,3.735581e+04,1.280540e+06,0.029172
